# downloads

In [ ]:
! pip install geemap

In [ ]:
! pip install -U -q earthengine-api

In [ ]:
! pip install -U -q folium

# imports

In [ ]:
import ee
import folium
import geemap
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# location selection

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=5g6wu-An_TCYQFeKfzyDMNC7lDsVW8_qUdUWV8qvwww&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgXKNqJm2xyRCx5PT7FhAXWrNK3OBu7-eeAgHb3QmuQ0tWTXYghQr8

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [61]:
#adding rastors from earth engine to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer


#soecfy terrain area you want to segment (limited number of pixels)
bound_ = [[6.219817916110255,36.75729987187895]
,[6.421348372653224,36.75729987187895]
,[6.421348372653224,36.849939905468865]
,[6.219817916110255,36.849939905468865]
,[6.219817916110255,36.75729987187895]]

crs_transform = [30, 0, 128685, 0, -30, 4270815]

point_ = bound_[0]
point = ee.Geometry.Point(point_)
bound = ee.Geometry.Polygon(bound_)

#importing satelite imagery

In [ ]:
#importing the whole collection of satelite images
satlite_dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
#selecting the image with the best covrage for our location of interest
sat_image = satlite_dataset.filterBounds(point).sort('CLOUD_COVER').first()

In [ ]:
# adding sat image to map
viz_params = {
    "min":0,
    "max":3000,
    "bands":["B4","B3","B2"]
}

map1 = folium.Map(location=[point_[1],point_[0]],zoom_start=12)
map1.add_ee_layer(sat_image.clip(bound),viz_params,name="sat_image")

In [ ]:
map1

#k-means clustering of vegetation

In [ ]:
#converting the rastor into a numpy array
rastor = geemap.ee_to_numpy(sat_image,region=bound,default_value=0)

In [ ]:
w,h,f=rastor.shape
rastor = rastor.reshape((w*h,f))

In [ ]:
#convert to pandas data-frame
#each pixel is row and each band is a column
data = pd.DataFrame(rastor,columns=sat_image.bandNames().getInfo())
data = data.drop(["sr_aerosol","pixel_qa","radsat_qa"],axis=1) #drop unused bands

In [ ]:
#normlize data
data = minmax_scale(data)

In [ ]:
#pca data reduction
n_components=5
pca = PCA(n_components)
data_r = pd.DataFrame(pca.fit_transform(data))

In [ ]:
data_train = data_r.sample(frac=0.5)

In [ ]:
#applying k-means
n_clusters = 4
km = KMeans(n_clusters,random_state=0)
km.fit(data_r)
classes = km.predict(data_r)

In [ ]:
classes = classes.reshape((w,h))

In [ ]:
#classes coloring function
hue = {
    0:[82/255, 182/255, 154/255],
    1:[30/255, 96/255, 145/255],
    2:[217/255, 237/255, 146/255],
    3:[177/255, 167/255, 166/255]
}
f= lambda i:hue[i]

In [ ]:
#adding k-means segmentation to the map 
map1 = folium.Map(location=[point_[1],point_[0]],zoom_start=12)
map1.add_ee_layer(sat_image.clip(bound),viz_params,name="sat_image")
folium.raster_layers.ImageOverlay(classes,bounds=[[bound_[2][1],bound_[2][0]],[bound_[0][1],bound_[0][0]]],colormap=f,name="sklearn cluster").add_to(map1)
folium.LayerControl().add_to(map1)
display(map1)


In [129]:
classes_ = classes.reshape((w*h))

#satelite elevation data

In [ ]:
#importing the whole collection of satelite images
elevation_data = ee.Image("USGS/SRTMGL1_003")

In [62]:
#project elevation data to match vegetation grid resolution 
elevation_projected = elevation_data.reproject('EPSG:32632',crs_transform )

In [63]:
viz_ele={
    "min":0,
    "max":900,
    "palette":"blue, red"
}

map1 = folium.Map(location=[point_[1],point_[0]],zoom_start=12)
map1.add_ee_layer(sat_image.clip(bound),viz_params,name="sat_image")
map1.add_ee_layer(elevation_projected.clip(bound),viz_ele,name="elevation")
folium.LayerControl().add_to(map1)
display(map1)

In [64]:
elevation_projected  = geemap.ee_to_numpy(elevation_projected,region=bound,default_value=None)

In [130]:
elevation_projected_ = elevation_projected.reshape((w*h))

#satelite wind data 

In [94]:
#get weather data
weather = ee.ImageCollection("NOAA/GFS0P25").filterBounds(bound).first()

In [95]:
#getting wind data
wind_u = ee.Image(weather.select("u_component_of_wind_10m_above_ground")).reproject('EPSG:32632',crs_transform )
wind_v = ee.Image(weather.select("v_component_of_wind_10m_above_ground")).reproject('EPSG:32632',crs_transform )

In [96]:
wind_u = geemap.ee_to_numpy(wind_u,region=bound,default_value=0)
wind_v = geemap.ee_to_numpy(wind_v,region=bound,default_value=0)

In [128]:
wind_u = wind_u.reshape((w*h))
wind_v = wind_v.reshape((w*h))

# satelite moisture 

In [110]:
#moisture data 
moisture_data = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture").filterBounds(bound).first().select("smp")

In [105]:
moisture = moisture_data.reproject('EPSG:32632',crs_transform )

In [106]:
moisture = geemap.ee_to_numpy(moisture,region=bound,default_value=0)

In [124]:
moisture = moisture.reshape((w*h))

#wraping data

In [133]:
wraped_data = pd.DataFrame({'veg_class':classes_,'elevation':elevation_projected_,'wind_u':wind_u,'wind_v':wind_v,"moisture":moisture})

In [136]:
wraped_data

,veg_class,elevation,wind_u,wind_v,moisture
0,2,379,-0.13,1.56,0.873476
1,2,386,-0.13,1.56,0.873476
2,2,390,-0.13,1.56,0.873476
3,2,390,-0.13,1.56,0.873476
4,2,392,-0.13,1.56,0.873476
...,...,...,...,...,...
219595,2,240,0.25,1.22,0.819649
219596,2,237,0.25,1.22,0.819649
219597,2,243,0.25,1.22,0.819649
219598,2,252,0.25,1.22,0.819649
